In [1]:
import rasterio
import keplergl
import numpy as np
import osmnx as ox
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import matplotlib
import mapclassify
import json
import base64
import IPython
import plotly.express as px


plt.style.use('ggplot')
plt.rcParams.update({'font.size': 16, 'axes.labelweight': 'bold', 'figure.figsize': (6, 6), 'axes.edgecolor': '0.2'})

In [2]:
import geopandas as gpd

In [ ]:
# Loading date
forest_data = gpd.read_file("data/FADM_PROV_FOREST")
forest_data = forest_data.to_crs("EPSG:4326")    # I'm converting to a different coordinate reference system
forest_data.head()

In [ ]:
type(forest_data)
forest_data.info()

In [ ]:
# Drop columns with all NaN values
forest_data_cleaned = forest_data.dropna(axis=1, how='all')

# Drop the specified columns
columns_to_drop = ['MP_BLCK_ID', 'EFF_DATE', 'RETRMNT_DT', 'FC_SKEY', 'OBJ_V_SKEY']
forest_data_cleaned = forest_data_cleaned.drop(columns=columns_to_drop)

forest_data_cleaned

In [ ]:
forest_data_cleaned.info()

In [ ]:
# Take a look at an example. The Kitimat Forest in BC. 
name = forest_data.iloc[0]["PRV_FRST_N"]
forest_data.iloc[[0]].plot(edgecolor="0.2", figsize=(10, 8), color = "green")
plt.title(name);

# 1. Basic Spatial Exploration

In [ ]:
# Take a look at the entire forest map.
forest_data.plot(edgecolor="0.2", figsize=(10, 8), color ="green")
plt.title("Forests in British Columbia");

# 2. Forest Area Analysis

In [ ]:
# Aggregate forest areas by region
region_area = forest_data.groupby('PRV_FRST_N')['AREA_SQM'].sum().sort_values(ascending=True)
region_area

In [ ]:
# Convert the region_area Series to a DataFrame for easier plotting
region_area_df = region_area.reset_index()

# Create a horizontal bar plot using Plotly
fig = px.bar(
    region_area_df,
    x='AREA_SQM',
    y='PRV_FRST_N',
    orientation='h',  # Horizontal bar chart
    title='Total Forest Area by Region',
    labels={'PRV_FRST_N': 'Forest Name', 'AREA_SQM': 'Area (square meters)'},
    height=1200,  # Set height to accommodate all bars
)

# Customize the layout
fig.update_layout(
    xaxis_title="Area (square meters)",
    yaxis_title="Forest Name",
    font=dict(size=10),
    margin={"r":0,"t":40,"l":0,"b":0}
)

# Show the figure
fig.show()

# 3. Create a base map for visualizing the polygons

In [ ]:
fig = px.choropleth_mapbox(
    forest_data_cleaned, 
    geojson=forest_data_cleaned.geometry, 
    locations=forest_data_cleaned.index,  # Use the index to bind the polygons
    hover_name="PRV_FRST_N",  # Show forest name on hover
    hover_data=["FEATURE_ID"],  # Additional data on hover
    mapbox_style="open-street-map",
    zoom=4,
    center={"lat": 53.7267, "lon": -127.6476},  # Center the map over British Columbia
    opacity=0.5,
    color_discrete_sequence=["green"]  # Set a fixed color green for all polygons
)

# Update layout
fig.update_layout(
    mapbox_accesstoken='your-mapbox-access-token-here',  # Optional: Use your Mapbox token for better styles
    title="Forest Data Visualization in BC",
    margin={"r":0,"t":0,"l":0,"b":0}
)

# Show the figure
fig.show()

# 4. Simplify the Geo data for quickly loading and visualization. 

In [ ]:
import geopandas as gpd
from shapely.geometry import Polygon

# Function to simplify geometries
def simplify_geometry(geom, tolerance=0.01):
    if geom.is_empty:
        return geom
    return geom.simplify(tolerance)

# Simplify geometries
forest_data_cleaned['geometry'] = forest_data_cleaned['geometry'].apply(lambda geom: simplify_geometry(geom, tolerance=0.01))

# Save the simplified GeoDataFrame to a file
forest_data_cleaned.to_file("data/FADM_PROV_FOREST_simplified.geojson", driver="GeoJSON")

print("Simplified data saved successfully.")

In [ ]:
forest_data_cleaned

In [ ]:

simplified_fig = px.choropleth_mapbox(
    forest_data_cleaned, 
    geojson=forest_data_cleaned.geometry, 
    locations=forest_data_cleaned.index,  # Use the index to bind the polygons
    hover_name="PRV_FRST_N",  # Show forest name on hover
    hover_data=["FEATURE_ID"],  # Additional data on hover
    mapbox_style="open-street-map",
    zoom=4,
    center={"lat": 53.7267, "lon": -127.6476},  # Center the map over British Columbia
    opacity=0.5,
    color_discrete_sequence=["green"]  # Set a fixed color green for all polygons
)

# Update layout
simplified_fig.update_layout(
    mapbox_accesstoken='your-mapbox-access-token-here',  # Optional: Use your Mapbox token for better styles
    title="Forest Data Visualization in BC",
    margin={"r":0,"t":0,"l":0,"b":0}
)

# Show the figure
simplified_fig.show()

# Loading the 2024 BC wildfire data

In [ ]:
# Loading date
wildfire_data = gpd.read_file("data/PROT_CURRENT_FIRE_PNTS_SP")
wildfire_data = wildfire_data.to_crs("EPSG:4326")
wildfire_data.head()

In [ ]:
# Wildfires by year
wildfires_by_year = wildfire_data['FIRE_YEAR'].value_counts().sort_index()
wildfires_by_year.plot(kind='bar', title='Number of Wildfires by Year')
plt.ylabel('Number of Wildfires')
plt.show()

In [ ]:
# Wildfires by cause
wildfires_by_cause = wildfire_data['FIRE_CAUSE'].value_counts()
wildfires_by_cause.plot(kind='bar', title='Number of Wildfires by Cause')
plt.ylabel('Number of Wildfires')
plt.show()

In [ ]:
# Take a look at the entire forest map.
wildfire_data.plot(edgecolor="0.2", figsize=(10, 8), color ="green")
plt.title("Wildfires in British Columbia in 2023 and 2024");

In [ ]:
filtered_data = wildfire_data

#filtered_data['Normalized_Size'] = filtered_data['SIZE_HA'] * 1000000000000/filtered_data['SIZE_HA'].max()

map_fig = px.scatter_mapbox(filtered_data,
                                lat="LATITUDE", lon="LONGITUDE",
                                #size="Normalized_Size",
                                color="FIRESTATUS",
                                hover_name="FIRE_NO",
                                hover_data={"SIZE_HA": True, "FIRESTATUS": True},
                                zoom=5,
                                height=500)

# Add forest data overlay
map_fig.add_trace(px.choropleth_mapbox(forest_data,
                                           geojson=forest_data.geometry,
                                           locations=forest_data.index,
                                           featureidkey="properties.FEATURE_ID",
                                           hover_name="PRV_FRST_N").data[0])

map_fig.update_layout(mapbox_style="open-street-map")
map_fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

map_fig